### Recently I published a self help book titled Inspiration: Thoughts on Spirituality, Technology, Wealth, Leadership and Motivation. The preview of the book can be read from the Amazon link https://lnkd.in/gj7bMQA

In this Kernel we will be covering following things

1.Data Preprocessing

2.Data Vizualizaion

3.Feature Engineering

4.Predicting Heart Disease

5.Conclusion

You can refer to my other notebooks from https://www.kaggle.com/binuthomasphilip/code

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1.Importing and Data Exploration data

### Importing Python Modules

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil import parser
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.svm import SVC 
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import GridSearchCV
import pickle
from lightgbm import LGBMClassifier
import warnings
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')

### Importing Data

In [ ]:

data=pd.read_csv('../input/heart-disease-uci/heart.csv')
data.head()

### Data contains following information

age: The person's age in years

sex: The person's sex (1 = male, 0 = female)

cp: The chest pain experienced (Value 1: typical angina, Value 2: atypical angina, Value 3: non-anginal pain, Value 4: asymptomatic)

trestbps: The person's resting blood pressure (mm Hg on admission to the hospital)

chol: The person's cholesterol measurement in mg/dl

fbs: The person's fasting blood sugar (> 120 mg/dl, 1 = true; 0 = false)

restecg: Resting electrocardiographic measurement (0 = normal, 1 = having ST-T wave abnormality, 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria)

thalach: The person's maximum heart rate achieved

exang: Exercise induced angina (1 = yes; 0 = no)

oldpeak: ST depression induced by exercise relative to rest ('ST' relates to positions on the ECG plot. See more here)

slope: the slope of the peak exercise ST segment (Value 1: upsloping, Value 2: flat, Value 3: downsloping)

ca: The number of major vessels (0-3)

thal: A blood disorder called thalassemia (3 = normal; 6 = fixed defect; 7 = reversable defect)

target: Heart disease (0 = no, 1 = yes)

### Shape of Data

In [ ]:
data.shape

### Missing Values

In [ ]:
data.isnull().sum()

We are very lucky here there are no Null Values in the dataset.But in this data the missing values are present in the form of value ZERO. Our next task would be to find out numbers of ZEROS in each column.

### Finding out Zero's

In [ ]:
cols = data.columns
cols

In [ ]:
print("# Rows in the dataset {0}".format(len(data)))
print("---------------------------------------------------")
for col in cols:
    print("# Rows in {1} with ZERO value: {0}".format(len(data.loc[data[col] ==0]),col))

The columns which have categorical values can have ZERO values.But columns like cp,trestbps,chol,fbs,exang,oldpean and Slope should not have value ZERO.The presence of ZERO in this columns indicate the presence of null values.

### Data Types

In [ ]:
data.dtypes

As all the columns are either integer or float there is no need to convert categorical values numeric values.

### Summary Of Data Set

In [ ]:
print('Rows     :',data.shape[0])
print('Columns  :',data.shape[1])
print('\nFeatures :\n     :',data.columns.tolist())
print('\nMissing values    :',data.isnull().values.sum())
print('\nUnique values :  \n',data.nunique())

### Describing the DataSet

In [ ]:
data.describe().T

1.From the above Stats we can conclude that the Mean age of patients in datset is 54.3 years

2.There are more men in the Dataset 68%

3.Mean Resting Blood Pressure value is 132 mm of Hg with a min of 94mm and maximum of 200mm

4.Mean Cholesterol level is 246 mg/dl with a mainimum value of 126 mg/dl and maximum value of 564 mg/dl

Histogram

### 1.From the above Stats we can conclude that the Mean age of patients in datset is 54.3 years

2.There are more men in the Dataset 68%

3.Mean Resting Blood Pressure value is 132 mm of Hg with a min of 94mm and maximum of 200mm

4.Mean Cholesterol level is 246 mg/dl with a mainimum value of 126 mg/dl and maximum value of 564 mg/dl

# 2.Data Visualisation

### Histogram

In [ ]:
data.hist(figsize = (12,12))
plt.show()

We can see that

1.Age,cholestrol,max heart rate,resting blood pressure and st_depression are numerical variables.

2.Chest pain type,exercie induced angina,fasting blood sugar,number major vessels,rest ecg,sex,st slope,target and thalassemia are categorical variables.

### Heart Disease Distribution in Dataset

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
data['target'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Rate of Heart Disease')
ax[0].set_ylabel('Count')
sns.countplot('target',data=data,ax=ax[1],order=data['sex'].value_counts().index)
ax[1].set_title('Rate of Heart Disease')
plt.show()

Data Set has 54 % cases where the Patient has Heart Disease

### Distribution of Parameters

In [ ]:
#Creating subplots
plt.figure(figsize=(15, 12))
#Subplot1 
plt.subplot(2,2,1)
plt.title('Distribution of Age')
sns.distplot(data['age'], rug = True)

#Subplot2
plt.subplot(2,2,2)
plt.title('Distribution of Resting Blood Pressure')
sns.distplot(data['trestbps'], rug = True)

#Subplot3
plt.subplot(2,2,3)
plt.title('Distribution of Cholesterol')
sns.distplot(data['chol'], rug = True)


#Subplot4
plt.subplot(2,2,4)
plt.title('Distribution of Max Hear Rate')
sns.distplot(data['thalach'], rug = True)
plt.ioff()
#plt.show()

Above plots show the distribution of all the key parameters resbonsible for heart health.This are all close to Gaussian Distribution.

### Chest Pain Type 

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
data['cp'].value_counts().plot.pie(explode=[0,0.05,0.05,0.05],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Chest Pain Type')
ax[0].set_ylabel('Count')
sns.countplot('cp',data=data,ax=ax[1],order=data['cp'].value_counts().index)
ax[1].set_title('Chest Pain Type')
plt.show()

Value 0: typical angina

Value 1: atypical angina

Value 2: non-anginal pain

Value 3: asymptomatic

### Heart Disease Based on Sex

In [ ]:
sns.barplot(x="sex",y ='age',hue ='target',data=data)
pass


We can see that more men have heart issues than women.

Men dont show emotions keep everything in their heart and finally suffer with heart diease.

Hormones also play a big role in all this.

### Cat plot

In [ ]:
sns.catplot(x='cp',y='target',kind='point',data=data,col='sex')
pass

We can see that women with chest pain type 3 have more chance of heart disease.

Men with chest pain type 1 have more chance of heart disease.

### Vilon Plots

In [ ]:

fig,ax = plt.subplots(nrows=4, ncols=2, figsize=(18,18))
plt.suptitle('Violin Plots',fontsize=24)
sns.violinplot(x="ca", data=data,ax=ax[0,0],palette='Set3')
sns.violinplot(x="trestbps", data=data,ax=ax[0,1],palette='Set3')
sns.violinplot (x ='chol', data=data, ax=ax[1,0], palette='Set3')
sns.violinplot(x='fbs', data=data, ax=ax[1,1],palette='Set3')
sns.violinplot(x='restecg', data=data, ax=ax[2,0], palette='Set3')
sns.violinplot(x='thalach', data=data, ax=ax[2,1],palette='Set3')
sns.violinplot(x='exang', data=data, ax=ax[3,0],palette='Set3')
sns.violinplot(x='age', data=data, ax=ax[3,1],palette='Set3')
plt.show()

Above Violin Plots show the range of Critcal Medical Parameters for Patients.Few of the observations are

1.Most people are in the category zero typical angina

2.Resting blood pressure for most people is around 130 bpm

3.Maximum heart rate is around 160 bpm.

### Vilon Plots Based on Sex

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
sns.violinplot("sex","trestbps",hue="target", data=data,split=True)
plt.subplot(2,2,2)
sns.violinplot("sex","chol",hue="target", data=data,split=True)
plt.subplot(2,2,3)
sns.violinplot("sex","thalach",hue="target", data=data,split=True)
plt.subplot(2,2,4)
sns.violinplot("sex","fbs",hue="target", data=data,split=True)
#ax[0].set_title('Sex and trestbps vs target')
#ax[0].set_yticks(range(0,110,10))
#sns.violinplot("Sex","Age", hue="Survived", data=data,split=True,ax=ax[1])
#ax[1].set_title('Sex and Age vs Survived')
#ax[1].set_yticks(range(0,110,10))
plt.ioff()
plt.show()

From the above Violin Plots we can see that resting blood pressure,cholesterol and maximum heart rate have an effect on the cause of heart Disease.In fact all this parameters are directly correlated.Surprisingly people with higher fasting sugar levels dont seem to be affected with heart disease.

### Effect of body parameters on Heart Disease based on Age

In [ ]:
pd.crosstab(data.age,data.target).plot(kind='bar',figsize=(20,6))
plt.title('Heart Disease Vs Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.savefig('HeartDiseaseAge.png')
plt.ioff()

We can clearly see that heart disease strikes more in early 40s and early 50s.Now to get a better picture we can plot the distribution of people with heart disease and their age.

### Age  Vs Heart Disease 

In [ ]:
# Bar chart for age with sorted index
# Changing title fontsize -> We have to use alternative matplotlib set_title function
plot = data[data.target == 1].age.value_counts().sort_index().plot(kind = "bar", figsize=(15,5), fontsize = 15)
plot.set_title("Heart disease: Age distribution", fontsize = 15)
plt.ioff()

We can see that heart disease frequency increases once a person is above 40 Years old.Age group of 41-45 and 51-55 have high chance of having heart problem.

### Effect of body parameters on Heart Disease based on Age

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.scatter(x=data.age[data.target==1],y=data.thalach[data.target==1],c='red')
plt.scatter(x=data.age[data.target==0],y=data.thalach[data.target==0],c='green')
plt.xlabel('Age')
plt.ylabel('Max Heart Rate')
plt.legend(['Disease','No Disease'])

plt.subplot(1,3,2)
plt.scatter(x=data.age[data.target==1],y=data.chol[data.target==1],c='red')
plt.scatter(x=data.age[data.target==0],y=data.chol[data.target==0],c='green')
plt.xlabel('Age')
plt.ylabel('Cholesterol')
plt.legend(['Disease','No Disease'])

plt.subplot(1,3,3)
plt.scatter(x=data.age[data.target==1],y=data.trestbps[data.target==1],c='red')
plt.scatter(x=data.age[data.target==0],y=data.trestbps[data.target==0],c='green')
plt.xlabel('Age')
plt.ylabel('Resting Blood Pressure')
plt.legend(['Disease','No Disease'])

plt.tight_layout()
plt.ioff()

1.We can see that higher heart rate at a younger age can be more dangerous.

2.Higher cholesterol is dangerous irrespective of age

3.Resting Blood Pressure doesnt have a clear correlation to Heart Disease.

### Heat Map

In [ ]:
corrmat = data.corr()
fig = plt.figure(figsize = (16,16))
sns.heatmap(corrmat,vmax = 1,square = True,annot = True,vmin = -1)
plt.show()

We can see that there is not much correlation between the features in the dataset.If corelation was high we can face issue of multicollinearity.In that case we would need to use feature engineering to avoid multi colinearity.

# 3.Feature Engineering

In [ ]:
cols

In [ ]:
final_cols1 = cols
final_cols1 = list(final_cols1)
#final_cols
list_to_remove = ['ca','cp','exang','fbs','restecg','sex','slope','target','thal']

final_cols = list(set(final_cols1) - set(list_to_remove))
final_cols

We have dropped the columns with categorical columns.This is because when we do feature engineering we will be repacing the numerical column Zero values with the mean values.This is not needed for the columns with categorical variables.

### Matrix of Features

In [ ]:
X = data.drop('target',axis=1) #predictor feature columns
y = data.target
y.value_counts()

We can see that the dataset is unbalanced.First we will balance the dataset.Balancing data set is a good approach to improve the accuracy of the machine learning model.

### Balancing Dataset

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_res_OS , Y_res_OS = sm.fit_resample(X,y)
pd.Series(Y_res_OS).value_counts()

### Numpy Arrat to Dataframe Conversion

In [ ]:
X_res_OS = pd.DataFrame(X_res_OS,columns=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal'])
Y_res_OS = pd.DataFrame(Y_res_OS,columns=['target'])

### Test Train Split 

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_res_OS,Y_res_OS,test_size = 0.1,random_state=10)
print('Training Set :',len(X_train))
print('Test Set :',len(X_test))
print('Training labels :',len(y_train))
print('Test labels :',len(y_test))

The number of rows of data is very low in the dataset.This may not be sufficient to build a good model.Let us see how our model works out.

### Replacing Zero Values in Numerical Feature with Mean of Column

In [ ]:
final_cols

In [ ]:
from sklearn.impute import SimpleImputer 
fill = SimpleImputer(missing_values=np.nan, strategy='mean')

X_train = fill.fit_transform(X_train[final_cols])
X_test = fill.fit_transform(X_test[final_cols])

### Important Features

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators = 1000,random_state = 123)
columns = ['target']
#admt.drop(columns, inplace=True, axis=1)
#X = admt.drop('target',axis = 1)
#y = admt['target']
#X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = .25,random_state = 123)
rf_model = RandomForestRegressor(n_estimators = 1000,random_state = 123)
rf_model.fit(X_train,y_train)
feature_importance = pd.DataFrame(sorted(zip(rf_model.feature_importances_, X.columns)), columns=['Value','Feature'])
plt.figure(figsize=(10, 6))
sns.barplot(x="Value", y="Feature", data=feature_importance.sort_values(by="Value", ascending=False))
plt.xlabel('Value',fontsize=20)
plt.ylabel('Feature',fontsize=20)
plt.title('Random Forest Feature Importance',fontsize=25)
plt.grid()
plt.ioff()
plt.tight_layout()

We can see Age,Cholosterol and Chest Pain have highest importance while predicting Heart disease.

# 4.Predictig Heart Disease 

### Grid Search 

In [ ]:
def FitModel(X_train,y_train,X_test,y_test,algo_name,algorithm,gridSearchParams,cv):
    np.random.seed(10)
    
    grid = GridSearchCV(
         estimator = algorithm,
         param_grid = gridSearchParams,
         cv=cv,scoring='accuracy',verbose=1,n_jobs=-1)
        
    grid_result = grid.fit(X_train,y_train)
    best_params = grid_result.best_params_
    pred = grid_result.predict(X_test)
    cm = confusion_matrix(y_test,pred)
    print(pred)
    
    print('Best Params :',best_params)
    print('Classification Report :',classification_report(y_test,pred))
    print('Accuracy Score :'+ str(accuracy_score(y_test,pred)))
    print('Confusion Matrix : \n',cm)

### Logistic Regression

In [ ]:
# Creating Regularization penalty
penalty = ['l1','l2']

# Create regularization hyperparameter space 
C = np.logspace(0,4,10)

# Create hyperparameter options 
hyperparameters = dict(C=C,penalty = penalty)

FitModel(X_train,y_train,X_test,y_test,'LogisticRegression',LogisticRegression(),hyperparameters,cv=5)

### XGBoost

In [ ]:
param = {
           'n_estimators':[100,500,1000,1500,2000],
           'max_depth':[2,3,4,5,6,7],
           'learning_rate':np.arange(0.01,0.1,0.01).tolist()
         }
FitModel(X_train,y_train,X_test,y_test,'XGBoost',XGBClassifier(),param,cv=5)

### Random Forest

In [ ]:
param = {
           'n_estimators':[100,500,1000,1500,2000],
           'max_depth':[2,3,4,5,6,7],
        
         }
FitModel(X_train,y_train,X_test,y_test,'Random Forest',RandomForestClassifier(),param,cv=5)

### SVC

In [ ]:
param = {
           'C':[0.1,1,100,1000],
           'gamma':[0.0001,0.001,0.005,0.1,1,3,5],
           
         }
FitModel(X_train,y_train,X_test,y_test,'SVC',SVC(),param,cv=5)

# 5.Conclusion
1.Most people fall in category 0 ie typical angina for chest pain

2.Resting blood pressure for most people is around 130 bpm.Resting Blood Pressure doesnt have a clear correlation to Heart Disease

3.Surprisingly people with higher fasting sugar levels dont seem to be affected with heart disease

4.We can clearly see that heart disease strikes more in early 40s and early 50s.Age group of 41-45 and 51-55 have high chance of having heart problem.

5.Higher cholesterol is dangerous irrespective of age

6.We can see that higher heart rate at a younger age can be more dangerous.

7.We can see that women with chest pain type 3 have more chance of heart disease.Men with chest pain type 1 have more chance of heart disease.

8.More Men have Heart Issues than Women

9.We are getting the best accuracy in Predicting Heart Disease with XGGoost Algorithm

### You can refer to my other notebooks from https://www.kaggle.com/binuthomasphilip/code